In [1]:
import pandas as pd

files = [
    "../data/raw/master_dataset.csv",
    "../data/raw/ed/edstays.csv",
]

for f in files:
    try:
        df = pd.read_csv(f, nrows=5)
        print(f"{f}: OK")
    except:
        print(f"{f}: ERRO AO CARREGAR")


../data/raw/master_dataset.csv: OK
../data/raw/ed/edstays.csv: OK


In [3]:
ed = pd.read_csv("../data/raw/ed/edstays.csv")
print(ed.stay_id.is_unique)  # True esperado
print(ed.subject_id.isna().sum())
print(ed.hadm_id.isna().sum())  # PODE SER > 0 no ED bruto


True
0
222071


In [9]:
master = pd.read_csv("../data/raw/master_dataset.csv")
missing = ~master.stay_id.isin(ed.stay_id)
missing.sum()

0

In [10]:
master_clean = master[master.hadm_id.notna()]
print(master_clean.shape)


(203016, 120)


In [12]:
# ed = pd.read_csv("edstays.csv")
(ed.intime > ed.outtime).sum()  # deve ser zero

6

In [14]:
invalid_time = (master.charttime < master.intime).sum()
print(invalid_time)


AttributeError: 'DataFrame' object has no attribute 'charttime'

In [16]:
tri = pd.read_csv("../data/raw/ed/triage.csv")
print(tri.stay_id.isin(ed.stay_id).mean())  # deve ser quase 1.0

1.0


In [18]:
from glob import glob

all_labs = pd.concat([pd.read_csv(f) for f in glob("labs/*.csv")])
print(all_labs.stay_id.isin(ed.stay_id).mean())


ValueError: No objects to concatenate

In [19]:
master["stay_id"] = master["stay_id"].astype(str)
master["subject_id"] = master["subject_id"].astype(int)
master["hadm_id"] = master["hadm_id"].astype("Int64")

for col in ["intime", "outtime", "charttime"]:
    if col in master.columns:
        master[col] = pd.to_datetime(master[col], errors="coerce")


In [20]:
missing_report = (master.isna().mean()*100).sort_values(ascending=False)
print(missing_report.head(20))


deathtime                       98.917869
time_to_icu_transfer            92.490714
time_to_icu_transfer_hours      92.490714
intime_icu                      92.490714
dod                             85.065410
hadm_id                         52.241306
insurance                       52.241306
edouttime                       52.241306
edregtime                       52.241306
dischtime                       52.241306
admittime                       52.241306
next_ed_visit_time              48.343986
next_ed_visit_time_diff         48.343986
next_ed_visit_time_diff_days    48.343986
ethnicity                       31.747148
ed_pain_last                    11.676433
ed_o2sat_last                    6.843776
triage_pain                      6.358934
ed_temperature_last              6.125334
triage_temperature               5.508284
dtype: float64
